<a href="https://colab.research.google.com/github/sezerblt/dataset_repository/blob/main/dataset_03_f_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pyplot
from keras.datasets import fashion_mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

veriseti yükleme ve veri önişleme

In [ ]:
def getNormalizeDataset():
  X_train,Y_train=fashion_mnist.load_data()[0]
  X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
  X_train=X_train.astype("float32")
  X_TRAIN=X_train/255.0
  Y_TRAIN = to_categorical(Y_train)

  (X_test,Y_test)=fashion_mnist.load_data()[1]
  X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
  X_test=X_test.astype("float32")
  X_TEST=X_test/255.0
  Y_TEST = to_categorical(Y_test)
  return X_TRAIN,Y_TRAIN,X_TEST,Y_TEST


model oluşturma

In [ ]:
def setModel(inp_sh):
  conv2d_1=Conv2D(filters=32,
                  kernel_size=(3,3),
                  activation="relu",
                  padding="same",
                  kernel_initializer="he_uniform",
                  input_shape=(inp_sh[1],inp_sh[2],1)
  )
  max_pool=MaxPooling2D(pool_size=(2,2))
  flat=Flatten()
  dense_1=Dense(units=100,activation="relu",
                kernel_initializer="he_uniform")
  dense_2=Dense(units=10,activation="softmax")
  opt_comp=SGD(lr=0.01,momentum=0.9)

  model=Sequential()
  
  model.add(conv2d_1)
  model.add(max_pool)
  model.add(flat)
  model.add(dense_1)
  model.add(dense_2)
  model.compile(optimizer=opt_comp,
                loss="categorical_crossentropy",
                metrics=["accuracy"]
  )
  return model


modelin kullanarak performans ölçümü

In [ ]:
def evaluateModel(x_data,y_data,n_splits=5):
  scores_list=list()
  history_list=list()
  kfold=KFold(n_splits=n_splits,shuffle=True,random_state=1)
  for train_i,test_i in kfold.split(x_data):
    model=setModel(x_data.shape)
    x_train_i=x_data[train_i]
    y_train_i=y_data[train_i]
    x_test_i=x_data[test_i]
    y_test_i=y_data[test_i]

    history=model.fit(x_train_i,
                           y_train_i,
                           epochs=3,
                           batch_size=32,
                           validation_data=(x_test_i,y_test_i),
                           verbose=0
    )
    _,acc=model.evaluate(x_test_i,y_test_i,verbose=0)
    #print(f"score:{scr*100}")
    print(f"dogruluk orani: {(acc*100.0)}")

    scores_list.append(acc)
    history_list.append(history)
    return scores_list,history_list

veri setinin cross ve acc grafigini incele

In [ ]:
def summaryHistory(histories):
  for i in range(len(histories)):
    pyplot.subplot(211)
    pyplot.title("cross entropy loss")
    pyplot.plot(histories[i].history["loss"],
                color="blue",label="train loss")
    pyplot.plot(histories[i].history["val_loss"],
                color="red",label="test loss")
    
    pyplot.subplot(212)
    pyplot.title("Accuracy classification")
    pyplot.plot(histories[i].history["accuracy"],
                color="blue",label="train accuracy")
    pyplot.plot(histories[i].history["val_accuracy"],
                color="red",label="test accuracy")
  pyplot.show()

score performans ölçümü

In [ ]:
def summaryScore(scores):
  print(f"dogruluk degerleri : ortalama{mean(scores)*100} ,std:{std(scores)*100} ,n:{len(scores)}")
  pyplot.boxplot(scores)
  pyplot.show()

basla

In [ ]:
def run():
  x_train,y_train,x_test,y_test=getNormalizeDataset()
  scores , histories=evaluateModel(x_train,y_train,5)
  summaryHistory(histories)
  summaryScore(scores)


In [ ]:
def savedModel():
  x_train,y_train,x_test,y_test=getNormalizeDataset()
  model=setModel(x_train.shape)
  model.fit(x_train,y_train,epochs=3,batch_size=32)
  _, acc = model.evaluate(x_test, y_test, verbose=0)
  print("> %.3f" % (acc * 100.0))
  model.save("final_model.h5")
savedModel()

#####od olarak biçimlendirilmiştir



In [ ]:
# load train and test dataset
def load_dataset():
  # load dataset
  (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
  # reshape dataset to have a single channel
  trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
  testX = testX.reshape((testX.shape[0], 28, 28, 1))
  # one hot encode target values
  trainY = to_categorical(trainY)
  testY = to_categorical(testY)
  return trainX, trainY, testX, testY

def prep_pixels(train, test):
  # convert from integers to floats
  train_norm = train.astype("float32")
  test_norm = test.astype("float32")
  # normalize to range 0-1
  train_norm = train_norm / 255.0
  test_norm = test_norm / 255.0
  # return normalized images
  return train_norm, test_norm

def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_uniform",
  input_shape=(28, 28, 1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation="relu", kernel_initializer="he_uniform"))
  model.add(Dense(10, activation="softmax"))
  # compile model
  opt = SGD(lr=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  return model

def evaluate_model(dataX, dataY, n_folds=5):
  scores, histories = list(), list()
  # prepare cross validation
  kfold = KFold(n_folds, shuffle=True, random_state=1)
  # enumerate splits
  for train_ix, test_ix in kfold.split(dataX):
    model = define_model()
    # select rows for train and test
    trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix],dataY[test_ix]
    history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX,testY), verbose=0)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
  print("> %.3f" % (acc * 100.0))
  # append scores 
  scores.append(acc)
  histories.append(history)
  return scores, histories

def summarize_diagnostics(histories):
  for i in range(len(histories)):
    # plot loss
    pyplot.subplot(211)
    pyplot.title("Cross Entropy Loss")
    pyplot.plot(histories[i].history["loss"], color="blue", label="train")
    pyplot.plot(histories[i].history["val_loss"], color="orange", label="test")
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title("Classification Accuracy")
    pyplot.plot(histories[i].history["acc"], color="blue", label="train")
    pyplot.plot(histories[i].history["val_acc"], color="orange", label="test")
  pyplot.show()

def summarize_performance(scores):
  # print summary
  print("Accuracy: mean=%.3f std=%.3f, n=%d" % (mean(scores)*100, std(scores)*100,
  len(scores)))
  # box and whisker plots of results
  pyplot.boxplot(scores)
  pyplot.show()


def run_test_harness():
  trainX, trainY, testX, testY = load_dataset()
  # prepare pixel data
  trainX, testX = prep_pixels(trainX, testX)
  # evaluate model
  scores, histories = evaluate_model(trainX, trainY)
  # learning curves
  summarize_diagnostics(histories)
  # summarize estimated performance
  summarize_performance(scores)

run_test_harness()

In [ ]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
import numpy
def load_image(filename):
  # load the image
  img = load_img(filename, grayscale=True, target_size=(28, 28))
  # convert to array
  img = img_to_array(img)
  # reshape into a single sample with 1 channel
  img = img.reshape(1, 28, 28, 1)
  # prepare pixel data
  img = img.astype("float32")
  img = img /255.0
  return img

def run_example():
  # load the image
  img = load_image("sample_image.png")
  # load model
  model = load_model("final_model.h5")
  # predict the class
  result = model.predict_classes(img)
  print(result[0])

run_example()